# Projeto 1 - Ciência dos Dados

Nome: Athur Fonseca

Nome: Caio Tieri

Nome: Leonardo Andrade

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
#import emoji

#.get_emoji_regexp

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\User\Desktop\INSPER\CIENCIA DE DADOS\PROJETO-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Bis 1-340.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)


,Treinamento,Nível de Relevância,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,➡️ nossa super pizza de banana nevada que é re...,1,NaN,NaN,NaN,NaN,NaN
1,comi os 2 ultimos bis que restavam tava ate es...,3,NaN,NaN,NaN,NaN,NaN
2,@luatenorinho bis fazendo jus ao nome toda vez...,4,NaN,NaN,NaN,1.0,Irrelevante
3,"a sheilla esconde o bis das filhas, socorro kk...",3,NaN,NaN,NaN,2.0,Neutro
4,não comprei uma caixa de bis é uma barra de ch...,2,NaN,NaN,NaN,3.0,Relevante


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Nível de Relevância,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,eu quero replay kkkk \nvamo de novo no bis 🤣🤣🤣...,1,NaN,NaN,NaN
1,@sbrinamendes juro acharam uma garrafa de cach...,2,NaN,1.0,Irrelevante
2,@aatsumiya sabia que você ia responder bis fã,1,NaN,2.0,Neutro
3,@anapaula_bis olá. queremos muito te ajudar. p...,1,NaN,3.0,Relevante
4,@gen_heleno mudaram as fraldas a esse monte de...,1,NaN,4.0,Essencial


___
## Classificador automático de sentimento


# Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.



O produto é um chocolate marcado por trazer um sentimento de insaciedade e vontade de comer sempre mais um.

# Para julgar a relevância dos tweets seguimos o critério abaixo:

1(Irrelevante): Não cita o produto ou cita o nome 'bis' mas se referindo a outra coisa.

2(neutro): Fala do produto mas este não se encaixa como o assunto principal do tweet

2(relevante): Cita o produto como assunto principal, expressa indiretamente uma opinião sobre o produto

4(muito relevante): Cita o produto  como assunto principal do tweet e expressa uma opinião clara sobre o mesmo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
treino = train.iloc[:,[0,1]]
treino.tail(20)

,Treinamento,Nível de Relevância
280,era só a caixinha de bis hoje🤤,3
281,a melhor parte foi o bis branco que mandaram d...,4
282,@bia_7285 resumindo teve a gatah brigando com ...,2
283,“ja que eu não posso fazer outra coisa vou com...,2
284,@koo__bis ele ts tao lindo,1
285,oxe cade meu bis q eu tinha guardado p come ca...,2
286,o bis eu ofereci pra turma na sala. mas os bom...,3
287,bis do pai\n\nmonstrooo,1
288,devorando uma caixa d bis enquanto choro,3
289,@dekulifws bis vc joga ff srs,1


In [7]:
#importando funções

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;|"]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    
def tiraarroba (string):
    palavras = []
    lista = string.split()
    for p in lista:
        if p[0] == '@':
            palavras.append(p)
    for palavra in palavras:
        lista.remove(palavra)
    limpo = ' '.join(lista)
    return limpo

def tiralink (string):
    palavras = []
    lista = string.split()
    for p in lista:
        if p[0] == 'h' and p[1] == 't' and p[2] == 't' and p[3] == 'p' and p[4] == 's':
            palavras.append(p)
    for palavra in palavras:
        lista.remove(palavra)
    limpo = ' '.join(lista)
    return limpo

# Removendo caracteres indesejados

In [8]:

treino['Treinamento'] = treino['Treinamento'].apply(cleanup)

<ipython-input-8-936eca644183>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treino['Treinamento'] = treino['Treinamento'].apply(cleanup)


In [9]:
#tira os @fulano citados

i=0

for t in treino.Treinamento:
   limpo = tiraarroba(t)
   treino.loc[i,'Treinamento'] = limpo
   i+=1
    
    


C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
#retira os links (https)

i=0

for t in treino.Treinamento:
   limpo = tiralink(t)
   treino.loc[i,'Treinamento'] = limpo
   i+=1
    

In [11]:
treino.head(50)

,Treinamento,Nível de Relevância
0,➡️ nossa super pizza de banana nevada que é re...,1
1,comi os 2 ultimos bis que restavam tava ate es...,3
2,bis fazendo jus ao nome toda vez que abro a caixa,4
3,a sheilla esconde o bis das filhas socorro kkk...,3
4,não comprei uma caixa de bis é uma barra de ch...,2
5,tou muito a sentir um bis do pai,1
6,dred bis do darwin nao acredito,1
7,meu idolo nunca foi preso dando fuga numa bis ...,1
8,meu cunhado trouxa bis pra mimmmmm,3
9,to vendo estrels,1


In [ ]:
def prob(frase, lista_tabela_relativa, lista_todas, todas_listas):
    probabilidade = 1
    a = 0
    t = 0
    count = 0
    y= 0
    lista2 = []
    multi = 1
    for tabela in lista_tabela_relativa:
        count = 0
        y +=1
        multi = 1
        for lista in todas_listas:
            for palavra in frase.split():
                if count >= len(frase.split()) :
                    break
                if palavra in tabela:
                    probabilidade = (1 + tabela[palavra])
                    count+=1
                else:
                    probabilidade = 1
                    count+=1
                t+=1
                multi *= probabilidade
        lista2.append(multi)
    if lista2.index(max(lista2)) == 0:
        return 'não relevante'
    elif lista2.index(max(lista2)) == 1:
        return 'pouco relevante'
    elif lista2.index(max(lista2)) == 2:
        return 'relevante'
    elif lista2.index(max(lista2)) == 3:
        return 'muito relevante'

print(prob('3d doritos boi', lista_tabela_relativas, lista_tudo,todas_listas))

In [ ]:
lista_fodasse = []
for a in test['Teste']:
    lista_fodasse.append( prob(limpa_frase(a), lista_tabela_relativas, lista_tudo, todas_listas))
    test['teste pc'] = lista_fodasse
test.head(40)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**